## Step for make it work

Install Redis for your OS
* pip install redis
* pip install python-redis-rate-limit # https://pypi.org/project/python-redis-rate-limit/
* pip install sparkly https://www.scivision.co/python-windows-visual-c++-14-required/
* pip install sparkly[redis]
* Install http://joeferner.github.io/redis-commander/  
Then `redis-commander --redis-host=localhost --redis-port=6379` from the browser http://localhost:8081/

In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys
sys.path.append("..")

In [26]:
from optimus import Optimus
op = Optimus()

In [4]:
# make some test data
columns = ['dogs', 'cats']
vals = [
    (2, 0),
    (0, 1),
    (4, 1)
]

# create DataFrame
df = op.spark.createDataFrame(vals, columns).repartition(1).cache()

In [5]:
df.table()

dogs 1 (bigint),cats 2 (bigint)
2,0
0,1
4,1


## Testing kombu and Rabbitmq

### Activate RabbitMQ GUI Managment
Reference https://www.youtube.com/watch?v=F4PvutsQJlc
Install erlang
Run C:\Program Files\erl10.0.1\Install.exe

Go to C:\Program Files\RabbitMQ Server\rabbitmq_server-3.7.7\sbin

`rabbitmq-plugins enable rabbitmq_management`

`rabbitmq-server.bat restart`

http://localhost:15672/

In [6]:
df.save.rabbit_mq("amqp://localhost:5672/","exchange_name", "example-queue","BOB"  )

In [12]:
df.save.mongo("localhost",27017, "db","BOB")

In [91]:
from pyspark.sql import functions as F

df_url = df.cols.nest(("http://google.com/",F.col("dogs"),"/",F.col("cats")),"url")

# send to mongo
df_url.enrinch("new_column", )

# Get info from mongo



dogs 1 (bigint),cats 2 (bigint),new 3 (string)
2,0,http://google.com/2/0
0,1,http://google.com/0/1
4,1,http://google.com/4/1


In [129]:
from optimus.enricher import Enricher

e = Enricher("localhost",27017, "db","BOB")
#e.run(df, )
e.show_cols("BOB")

cats
dogs
_id


In [120]:
print(e.db_exists("db"))
print(e.get_collection("BOB"))

True
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'db'), 'BOB')


NameError: name 'optimus' is not defined

In [35]:
import socket
import sys

host = '127.0.0.1'
port = 5005


s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((host,port))
s.send(b"1")  
s.close()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext




ssc  = StreamingContext(op.sc, 1)


lines = ssc.socketTextStream("localhost", 9998)

# Split each line into words
words = lines.flatMap(lambda line: line.split(" "))

# Count each word in each batch
pairs = words.map(lambda word: (word, 1))


wordCounts = pairs.reduceByKey(lambda x, y: x + y)

# Print the first ten elements of each RDD generated in this DStream to the console
wordCounts.pprint()


ssc.start()             # Start the computation
ssc.awaitTermination()  # Wait for the computation to terminate

-------------------------------------------
Time: 2018-09-06 17:28:44
-------------------------------------------

-------------------------------------------
Time: 2018-09-06 17:28:45
-------------------------------------------

